### 1.0 import libraries

In [1]:
import os
import random

import pyvista as pv
import trimesh as tm
import numpy as np
import topogenesis as tg

### 1.1 load macrovoxels

In [2]:
# load lattice CSV file
lattice_path = os.path.relpath('../data/macrovoxels.csv')
macro_lattice = tg.lattice_from_csv(lattice_path)

### 1.x generate a single random configuration to be tested

In [3]:
# create random configuration
# config_rand = [random.randint(0,1) for i in range(27)]
config_rand = [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]
print(config_rand)
# voxels of the configuration
config_index = [i for i in range(26) if config_rand[i] == 0] # random indices

config_rand = np.reshape(config_rand,(3,3,3))
config_rand = np.pad(config_rand, 1) # pad to regain original lattice structure


[0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]


In [4]:
# saving the random configuration for future use
random_lattice = tg.to_lattice(config_rand, macro_lattice)
save_path = os.path.relpath("../data/random_lattice.csv")
random_lattice.to_csv(save_path)

## Identifying the roof

In [5]:
# padding to avoid the rolling issue
random_lattice_padded = np.pad(random_lattice, 1, mode='constant',constant_values=0)

In [6]:
# shifting all the voxels one level down
shifted_lattice = np.roll(random_lattice_padded, (0,0,-1)) 

In [7]:
# an exposed roof surface exists where a voxel is filled (1) and the voxel above it is empty (0)
top_voxels_3darray_padded = (random_lattice_padded == 1) *  (shifted_lattice == 0)

In [8]:
# removing the pad
top_voxels_3darray = top_voxels_3darray_padded[1:-1, 1:-1, 1:-1]
# convert to lattice
top_voxels_lattice = tg.to_lattice(top_voxels_3darray, random_lattice)

In [9]:
# extracting the centroids of all top voxels
top_centroids = top_voxels_lattice.centroids

In [10]:
unit = random_lattice.unit
meshes= []

for i, cen in enumerate(top_centroids):
    # generating the vertices of the top face
    # centroid + half of the unit size in the four top directions
    v0 = cen + 0.5 * unit * np.array([ 1, 1, 1]) # top right front
    v1 = cen + 0.5 * unit * np.array([ 1,-1, 1]) # top right back
    v2 = cen + 0.5 * unit * np.array([-1,-1, 1]) # top left back
    v3 = cen + 0.5 * unit * np.array([-1, 1, 1]) # top left front

    face_a = [v0,v1,v2] # trimesh only takes triangular meshes, no quad meshes
    face_b = [v2,v3,v0]

    mesh_a = tm.Trimesh(vertices= face_a, faces= [[0,1,2]])
    mesh_b = tm.Trimesh(vertices= face_b, faces= [[0,1,2]])
    
    meshes.append(mesh_a)
    meshes.append(mesh_b)

### Merge meshes

In [11]:
# combined roof mesh
combined_roof = tm.util.concatenate(meshes)

In [12]:
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

### 1.x visuzalizing

In [13]:
p = pv.Plotter(notebook=True)

p.add_mesh(tri_to_pv(combined_roof), color='#abd8ff', opacity=0.9)
random_lattice.fast_vis(p)

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(204.88887394336027, 84.88887394336027, 144.88887394336027),
 (60.0, -60.0, 0.0),
 (0.0, 0.0, 1.0)]